In [6]:
!pip install beautifulsoup4 openpyxl requests pandas


In [ ]:
!pip install feedparser

import feedparser
import datetime
from dateutil.parser import parse

today = datetime.datetime.now()
today_diff = today + datetime.timedelta(days=-1)

feeds = {'1': '사건사고', '2': '보안컬럼', '5': '긴급경보', '6': '보안정책'}

for key, value in feeds.items():

    url = 'http://www.boannews.com/media/news_rss.xml?kind=' + key

    f = feedparser.parse(url)
    articles = f['entries']

    cnt_article = 0

    print("========================= RSS |", value, "=====================================", '\n')

    for article in articles:
        article_title = article['title']
        article_published = article['updated']
        article_link = article['link']
        article_summary = article['summary']

        article_date = parse(article_published)
        if article_date.date() >= today_diff.date():
            print(cnt_article + 1, '|', article_title)
            print(article_published)
            print(article_link)
            print(article_summary, '\n')
            cnt_article = cnt_article + 1

    if cnt_article == 0:
        print("해당 카테고리의 피드가 없습니다.", '\n')

    print("=============================================================================", '\n')

In [3]:
import feedparser
from datetime import datetime, timedelta
from dateutil import parser

# 수집 기간 설정 (예: 최근 1일간의 뉴스)
end_date = datetime.today()
start_date = end_date - timedelta(days=1)

# 카테고리 별 RSS URL 리스트 정의
rss_urls = [
    # 다음뉴스 종합
    'http://media.daum.net/rss/today/primary/all/rss2.xml',
    # 다음뉴스 연예
    'http://media.daum.net/rss/today/primary/entertain/rss2.xml',
    # 다음뉴스 스포츠
    'http://media.daum.net/rss/today/primary/sports/rss2.xml',
    # 다음뉴스 사회
    'http://media.daum.net/rss/part/primary/society/rss2.xml',
    # 다음뉴스 정치
    'http://media.daum.net/rss/part/primary/politics/rss2.xml',
    # 다음뉴스 경제
    'http://media.daum.net/rss/part/primary/economic/rss2.xml',
    # 다음뉴스 국제
    'http://media.daum.net/rss/part/primary/foreign/rss2.xml',
    # 다음뉴스 문화·생활
    'http://media.daum.net/rss/part/primary/culture/rss2.xml',
    # 다음뉴스 IT·과학
    'http://media.daum.net/rss/part/primary/digital/rss2.xml'
    # 다른 카테고리의 RSS URL도 필요한 경우 추가
]

def parse_rss(url):
    url=('https://news.google.com/rss?hl=ko&gl=KR&ceid=KR:ko')

    feed = feedparser.parse(url)
    print(feed)
    '''
    for entry in feed.entries:
        if hasattr(entry, 'published'):
            published = parser.parse(entry.published)
            if start_date <= published <= end_date:
                print(f"제목: {entry.title}")
                print(f"링크: {entry.link}")
                print(f"작성일: {published}")
                print(f"요약: {entry.summary}\n")
    '''
# 각 카테고리의 뉴스 수집 및 출력
parse_rss('http://media.daum.net/rss/part/primary/digital/rss2.xml')


{'bozo': 1, 'entries': [], 'feed': {'language': 'en-US', 'html': {'lang': 'ko', 'class': 'os_unknown none unknown version_0 '}, 'meta': {'http-equiv': 'X-UA-Compatible', 'content': 'IE=edge'}, 'links': [{'rel': 'shortcut icon', 'href': 'https://m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico', 'type': 'text/html'}, {'rel': 'stylesheet', 'type': 'text/css', 'href': 'https://t1.daumcdn.net/media/kraken/news/00fb0e7/style.css.merged.css'}, {'rel': 'stylesheet', 'type': 'text/css', 'href': 'https://t1.daumcdn.net/media/kraken/news/00fb0e7/calendar.css.merged.css'}], 'script': {'src': '//t1.daumcdn.net/media/kraken/news/00fb0e7/homeMain.merged.js'}, 'summary': '<div class="direct-link">\n    \n    <a href="https://news.daum.net/#mainContent">본문 바로가기</a>\n    <a href="https://news.daum.net/#gnbContent">메뉴 바로가기</a>\n</div>\n\n<div class="container-doc">\n<header class="doc-header">\n    \n    <div class="head_top">\n        <h1 class="doc-title">\n            <a class="link_daum" href="https

### 다음뉴스 크롤링 코드

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = "https://news.daum.net/breakingnews/digital?page="

# Define the number of pages you want to scrape, for instance 5 pages
num_pages = 40

# Create an empty DataFrame to store results
all_data = pd.DataFrame(columns=["Title", "Link"])

# Loop through pages
for page in range(1, num_pages + 1):
    # Construct full URL
    url = base_url + str(page)

    # Get the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract news items
    news_items = soup.select('li > div.cont_thumb > strong.tit_thumb')

    # Extract titles and links and append to the DataFrame
    for item in news_items:
        title = item.select_one('a.link_txt').text
        link = item.select_one('a.link_txt')['href']
        all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)

# Save DataFrame to Excel
file_name = "daum_news_multiple_pages.xlsx"
all_data.to_excel(file_name, index=False)

# If using Colab, download the Excel file
from google.colab import files
files.download(file_name)

# Display the first few rows of the DataFrame for a quick check
print(all_data.head())


<ipython-input-9-4e0a7258d331>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
<ipython-input-9-4e0a7258d331>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
<ipython-input-9-4e0a7258d331>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
<ipython-input-9-4e0a7258d331>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
<ipython

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

                               Title                                    Link
0      로보락, 현대홈쇼핑서 'S7 맥스 울트라' 할인 행사  https://v.daum.net/v/20230920220700991
1   네이버는 다르네…"비 오는 날 좋은 음식 뭐야?" 물었더니  https://v.daum.net/v/20230920220004907
2         ‘새만금 신재생에너지박람회’ 군산서 사흘간 개최  https://v.daum.net/v/20230920215848894
3  국립소방연구원 “전기차 화재 진압…골든타임 10분 사수해야”  https://v.daum.net/v/20230920215100794
4      전라북도 “한빛원전 수명 연장 전 주민 공론화 필요”  https://v.daum.net/v/20230920214221684
